In [1]:
# 📦 Required Libraries
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K

2025-06-24 19:07:22.322438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750792042.553757      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750792042.621716      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# ⚙️ Dice Loss Function
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    y_true_f = K.flatten(tf.cast(y_true, 'float32'))
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [6]:
# 📁 Data Directories
train_dir = "/kaggle/input/fake-image-detection-mobilenet/Train"
val_dir   = "/kaggle/input/fake-image-detection-mobilenet/Validation"
test_dir  = "/kaggle/input/fake-image-detection-mobilenet/Test"

In [7]:
# 📈 Data Preprocessing + Augmentation
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='binary')
val_generator   = val_datagen.flow_from_directory(val_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='binary')
test_generator  = test_datagen.flow_from_directory(test_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 750 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [9]:
# 📊 Steps per Epoch
steps_per_epoch   = int(np.ceil(train_generator.samples / batch_size))
validation_steps  = int(np.ceil(val_generator.samples / batch_size))

In [10]:
# 📦 MobileNetV2 Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze base model initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

2025-06-24 19:11:04.859566: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
# 🔧 Compile Model (Freeze Phase) with Dice Loss
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=dice_loss,
              metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])


In [13]:
# 📌 Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
checkpoint = ModelCheckpoint('best_mobilenetv2_dice.h5', monitor='val_loss', mode='min', save_best_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)

callbacks_list = [early_stop, checkpoint, lr_reduce]

In [14]:
# 📊 Initial Training (Freeze Phase)
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks_list
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.5172 - loss: 0.3505 - precision: 0.5170 - recall: 0.9903 - val_accuracy: 0.5000 - val_loss: 0.3463 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.4940 - loss: 0.3495 - precision: 0.4940 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.3436 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.5173 - loss: 0.3240 - precision: 0.5173 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.3422 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.5011 - loss: 0.3374 - precision: 0.5011 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.3395 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accu

In [15]:
# 🔓 Unfreeze last 30 layers of base model
for layer in base_model.layers[-30:]:
    layer.trainable = True


In [16]:
# 🔧 Compile Again (Fine-tuning Phase) with Dice Loss
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss=dice_loss,
              metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])


In [18]:
# 📊 Fine-tuning Training
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=25,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks_list
)

Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5164 - loss: 0.3259 - precision: 0.5057 - recall: 0.9981 - val_accuracy: 0.5333 - val_loss: 0.3160 - val_precision: 0.5172 - val_recall: 1.0000 - learning_rate: 1.0000e-05
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.6371 - loss: 0.2771 - precision: 0.5841 - recall: 0.9774 - val_accuracy: 0.6013 - val_loss: 0.2923 - val_precision: 0.5565 - val_recall: 0.9973 - learning_rate: 1.0000e-05
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6714 - loss: 0.2734 - precision: 0.6045 - recall: 0.9422 - val_accuracy: 0.6347 - val_loss: 0.2748 - val_precision: 0.5783 - val_recall: 0.9947 - learning_rate: 1.0000e-05
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.7269 - loss: 0.2370 - precision: 0.6572 - recall: 0.9350 - val_accuracy: 0.6533 - val_loss: 0.2622 - val_precision: 0.5914 - val_recall: 0.9920 - learning_rate: 1.0000e-05
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accu

KeyboardInterrupt: 

In [26]:

# 📈 Final Test Evaluation
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)

print("\n📊 Final Test Metrics:")
print(f"Test Accuracy:  {test_accuracy*100:.2f}%")
print(f"Test Precision: {test_precision*100:.2f}%")
print(f"Test Recall:    {test_recall*100:.2f}%")
print(f"Test Loss:      {test_loss:.4f}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 620ms/step - accuracy: 0.7418 - loss: 0.2153 - precision: 0.7008 - recall: 0.8947

📊 Final Test Metrics:
Test Accuracy:  73.50%
Test Precision: 68.08%
Test Recall:    88.50%
Test Loss:      0.2374


NameError: name 'train_acc' is not defined